In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches

# Set consistent style and parameters for all plots
plt.style.use('default')  # Use default style for consistency
sns.set_palette("tab10")  # Consistent color palette

# Global font and style settings
plt.rcParams.update({
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'font.size': 16,
    'axes.titlesize': 20,
    'axes.labelsize': 18,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 16,
    'figure.titlesize': 22,
    'lines.linewidth': 3,
    'lines.markersize': 10,
    'axes.linewidth': 1.5,
    'grid.linewidth': 1.0,
    'axes.grid': True,
    'grid.alpha': 0.3
})

# Consistent color scheme
COLORS = {
    'BNScale': '#1f77b4',      # Blue
    'MagnitudeL2': '#ff7f0e',  # Orange
    'Random': '#d62728',       # Red
    'MLP': '#2ca02c',          # Green
    'LSTM': '#9467bd',         # Purple
    'MobileNetV2': '#8c564b',  # Brown
    'ResNet-18': '#e377c2'     # Pink
}

def load_all_results():
    """Load all experimental results from JSON files"""
    try:
        with open('mobileNetV2_complete_results.json', 'r') as f:
            mobilenetv2_results = json.load(f)

        with open('resnet18_complete_results.json', 'r') as f:
            resnet18_results = json.load(f)

        with open('mlp_complete_results.json', 'r') as f:
            mlp_results = json.load(f)

        with open('lstm_complete_results.json', 'r') as f:
            lstm_results = json.load(f)

        return mobilenetv2_results, resnet18_results, mlp_results, lstm_results
    except FileNotFoundError as e:
        print(f"Error loading files: {e}")
        return None, None, None, None

def results_to_dataframe(results, model_name, task='classification'):
    """Convert results dictionary to pandas DataFrame"""
    data = []
    for strategy, strategy_results in results.items():
        for sparsity, metrics in strategy_results.items():
            row = {
                'model': model_name,
                'strategy': strategy,
                'sparsity': float(sparsity),
                'sparsity_percent': float(sparsity) * 100,
                'macs': metrics['macs'],
                'macs_millions': metrics['macs'] / 1e6,
                'params': metrics['params'],
                'params_millions': metrics['params'] / 1e6,
                'size_mb': metrics['size_mb'],
                'loss': metrics['loss']
            }

            if task == 'classification':
                row['accuracy'] = metrics['accuracy']
            else:  # regression
                row['mse'] = float(metrics.get('mse', metrics.get('loss', 0)))
                row['mae'] = float(metrics.get('mae', 0))

            data.append(row)

    return pd.DataFrame(data)

def create_output_directory():
    """Create output directory for plots"""
    os.makedirs('chapter5_plots', exist_ok=True)
    return 'chapter5_plots'

# ============================================================================
# Section 5.3.1: MobileNetV2 Performance Analysis
# ============================================================================

def plot_mobilenetv2_accuracy_vs_sparsity(mobilenet_df, output_dir):
    """Plot MobileNetV2 accuracy vs sparsity for all strategies"""
    fig, ax = plt.subplots(figsize=(12, 8))

    for strategy in sorted(mobilenet_df['strategy'].unique()):
        data = mobilenet_df[mobilenet_df['strategy'] == strategy].sort_values('sparsity')
        ax.plot(data['sparsity_percent'], data['accuracy'],
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('Accuracy (%)', fontweight='bold')
    ax.set_title('MobileNetV2: Accuracy vs Sparsity Level', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(84, 86.5)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/mobilenetv2_accuracy_vs_sparsity.png', bbox_inches='tight')
    plt.close()

def plot_mobilenetv2_macs_reduction(mobilenet_df, output_dir):
    """Plot MobileNetV2 MACs reduction"""
    fig, ax = plt.subplots(figsize=(12, 8))

    baseline_macs = mobilenet_df[mobilenet_df['sparsity'] == 0.0]['macs_millions'].iloc[0]

    for strategy in sorted(mobilenet_df['strategy'].unique()):
        data = mobilenet_df[mobilenet_df['strategy'] == strategy].sort_values('sparsity')
        macs_reduction = (1 - data['macs_millions'] / baseline_macs) * 100
        ax.plot(data['sparsity_percent'], macs_reduction,
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MACs Reduction (%)', fontweight='bold')
    ax.set_title('MobileNetV2: Computational Cost Reduction', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/mobilenetv2_macs_reduction.png', bbox_inches='tight')
    plt.close()

def plot_mobilenetv2_size_reduction(mobilenet_df, output_dir):
    """Plot MobileNetV2 model size reduction"""
    fig, ax = plt.subplots(figsize=(12, 8))

    baseline_size = mobilenet_df[mobilenet_df['sparsity'] == 0.0]['size_mb'].iloc[0]

    for strategy in sorted(mobilenet_df['strategy'].unique()):
        data = mobilenet_df[mobilenet_df['strategy'] == strategy].sort_values('sparsity')
        size_reduction = (1 - data['size_mb'] / baseline_size) * 100
        ax.plot(data['sparsity_percent'], size_reduction,
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('Model Size Reduction (%)', fontweight='bold')
    ax.set_title('MobileNetV2: Model Size Reduction', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/mobilenetv2_size_reduction.png', bbox_inches='tight')
    plt.close()

def plot_mobilenetv2_pareto_frontier(mobilenet_df, output_dir):
    """Plot MobileNetV2 Pareto frontier (Accuracy vs MACs)"""
    fig, ax = plt.subplots(figsize=(12, 8))

    for strategy in sorted(mobilenet_df['strategy'].unique()):
        data = mobilenet_df[mobilenet_df['strategy'] == strategy].sort_values('macs_millions')
        ax.scatter(data['macs_millions'], data['accuracy'],
                  color=COLORS[strategy], s=150, label=strategy, alpha=0.8,
                  edgecolors='black', linewidth=1.5)
        ax.plot(data['macs_millions'], data['accuracy'],
               color=COLORS[strategy], linestyle='--', alpha=0.7, linewidth=2)

    ax.set_xlabel('MACs (Millions)', fontweight='bold')
    ax.set_ylabel('Accuracy (%)', fontweight='bold')
    ax.set_title('MobileNetV2: Pareto Frontier (Accuracy vs Computational Cost)', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/mobilenetv2_pareto_frontier.png', bbox_inches='tight')
    plt.close()

# ============================================================================
# Section 5.3.2: ResNet-18 Performance Analysis
# ============================================================================

def plot_resnet18_accuracy_vs_sparsity(resnet_df, output_dir):
    """Plot ResNet-18 accuracy vs sparsity for all strategies"""
    fig, ax = plt.subplots(figsize=(12, 8))

    for strategy in sorted(resnet_df['strategy'].unique()):
        data = resnet_df[resnet_df['strategy'] == strategy].sort_values('sparsity')
        ax.plot(data['sparsity_percent'], data['accuracy'],
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('Accuracy (%)', fontweight='bold')
    ax.set_title('ResNet-18: Accuracy vs Sparsity Level', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(82, 85)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/resnet18_accuracy_vs_sparsity.png', bbox_inches='tight')
    plt.close()

def plot_resnet18_macs_reduction(resnet_df, output_dir):
    """Plot ResNet-18 MACs reduction"""
    fig, ax = plt.subplots(figsize=(12, 8))

    baseline_macs = resnet_df[resnet_df['sparsity'] == 0.0]['macs_millions'].iloc[0]

    for strategy in sorted(resnet_df['strategy'].unique()):
        data = resnet_df[resnet_df['strategy'] == strategy].sort_values('sparsity')
        macs_reduction = (1 - data['macs_millions'] / baseline_macs) * 100
        ax.plot(data['sparsity_percent'], macs_reduction,
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MACs Reduction (%)', fontweight='bold')
    ax.set_title('ResNet-18: Computational Cost Reduction', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/resnet18_macs_reduction.png', bbox_inches='tight')
    plt.close()

def plot_resnet18_pareto_frontier(resnet_df, output_dir):
    """Plot ResNet-18 Pareto frontier (Accuracy vs MACs)"""
    fig, ax = plt.subplots(figsize=(12, 8))

    for strategy in sorted(resnet_df['strategy'].unique()):
        data = resnet_df[resnet_df['strategy'] == strategy].sort_values('macs_millions')
        ax.scatter(data['macs_millions'], data['accuracy'],
                  color=COLORS[strategy], s=150, label=strategy, alpha=0.8,
                  edgecolors='black', linewidth=1.5)
        ax.plot(data['macs_millions'], data['accuracy'],
               color=COLORS[strategy], linestyle='--', alpha=0.7, linewidth=2)

    ax.set_xlabel('MACs (Millions)', fontweight='bold')
    ax.set_ylabel('Accuracy (%)', fontweight='bold')
    ax.set_title('ResNet-18: Pareto Frontier (Accuracy vs Computational Cost)', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/resnet18_pareto_frontier.png', bbox_inches='tight')
    plt.close()

# ============================================================================
# Section 5.3.3: CNN Comparative Analysis
# ============================================================================

def plot_cnn_accuracy_comparison(mobilenet_df, resnet_df, output_dir):
    """Compare CNN accuracies using MagnitudeL2 strategy"""
    fig, ax = plt.subplots(figsize=(12, 8))

    # MobileNetV2 data
    mobilenet_data = mobilenet_df[mobilenet_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    ax.plot(mobilenet_data['sparsity_percent'], mobilenet_data['accuracy'],
            'o-', color=COLORS['MobileNetV2'], label='MobileNetV2', linewidth=3, markersize=10)

    # ResNet-18 data
    resnet_data = resnet_df[resnet_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    ax.plot(resnet_data['sparsity_percent'], resnet_data['accuracy'],
            'o-', color=COLORS['ResNet-18'], label='ResNet-18', linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('Accuracy (%)', fontweight='bold')
    ax.set_title('CNN Architectures: Accuracy Comparison (MagnitudeL2 Pruning)', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/cnn_accuracy_comparison.png', bbox_inches='tight')
    plt.close()

def plot_cnn_efficiency_comparison(mobilenet_df, resnet_df, output_dir):
    """Compare CNN efficiency (MACs reduction)"""
    fig, ax = plt.subplots(figsize=(12, 8))

    # Calculate MACs reduction for both models
    mobilenet_baseline = mobilenet_df[mobilenet_df['sparsity'] == 0.0]['macs_millions'].iloc[0]
    resnet_baseline = resnet_df[resnet_df['sparsity'] == 0.0]['macs_millions'].iloc[0]

    mobilenet_data = mobilenet_df[mobilenet_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    mobilenet_reduction = (1 - mobilenet_data['macs_millions'] / mobilenet_baseline) * 100

    resnet_data = resnet_df[resnet_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    resnet_reduction = (1 - resnet_data['macs_millions'] / resnet_baseline) * 100

    ax.plot(mobilenet_data['sparsity_percent'], mobilenet_reduction,
            'o-', color=COLORS['MobileNetV2'], label='MobileNetV2', linewidth=3, markersize=10)
    ax.plot(resnet_data['sparsity_percent'], resnet_reduction,
            'o-', color=COLORS['ResNet-18'], label='ResNet-18', linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MACs Reduction (%)', fontweight='bold')
    ax.set_title('CNN Architectures: Computational Efficiency Comparison', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/cnn_efficiency_comparison.png', bbox_inches='tight')
    plt.close()

# ============================================================================
# Section 5.4.1: MLP Performance Analysis
# ============================================================================

def plot_mlp_mse_vs_sparsity(mlp_df, output_dir):
    """Plot MLP MSE vs sparsity"""
    fig, ax = plt.subplots(figsize=(12, 8))

    for strategy in sorted(mlp_df['strategy'].unique()):
        data = mlp_df[mlp_df['strategy'] == strategy].sort_values('sparsity')
        ax.plot(data['sparsity_percent'], data['mse'],
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MSE', fontweight='bold')
    ax.set_title('MLP: Mean Squared Error vs Sparsity Level', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/mlp_mse_vs_sparsity.png', bbox_inches='tight')
    plt.close()

def plot_mlp_mae_vs_sparsity(mlp_df, output_dir):
    """Plot MLP MAE vs sparsity"""
    fig, ax = plt.subplots(figsize=(12, 8))

    for strategy in sorted(mlp_df['strategy'].unique()):
        data = mlp_df[mlp_df['strategy'] == strategy].sort_values('sparsity')
        ax.plot(data['sparsity_percent'], data['mae'],
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MAE', fontweight='bold')
    ax.set_title('MLP: Mean Absolute Error vs Sparsity Level', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/mlp_mae_vs_sparsity.png', bbox_inches='tight')
    plt.close()

def plot_mlp_macs_reduction(mlp_df, output_dir):
    """Plot MLP MACs reduction"""
    fig, ax = plt.subplots(figsize=(12, 8))

    baseline_macs = mlp_df[mlp_df['sparsity'] == 0.0]['macs'].iloc[0] / 1000  # Convert to thousands

    for strategy in sorted(mlp_df['strategy'].unique()):
        data = mlp_df[mlp_df['strategy'] == strategy].sort_values('sparsity')
        macs_reduction = (1 - (data['macs'] / 1000) / baseline_macs) * 100
        ax.plot(data['sparsity_percent'], macs_reduction,
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MACs Reduction (%)', fontweight='bold')
    ax.set_title('MLP: Computational Cost Reduction', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/mlp_macs_reduction.png', bbox_inches='tight')
    plt.close()

# ============================================================================
# Section 5.4.2: LSTM Performance Analysis
# ============================================================================

def plot_lstm_mse_vs_sparsity(lstm_df, output_dir):
    """Plot LSTM MSE vs sparsity"""
    fig, ax = plt.subplots(figsize=(12, 8))

    for strategy in sorted(lstm_df['strategy'].unique()):
        data = lstm_df[lstm_df['strategy'] == strategy].sort_values('sparsity')
        ax.plot(data['sparsity_percent'], data['mse'],
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MSE', fontweight='bold')
    ax.set_title('LSTM: Mean Squared Error vs Sparsity Level', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/lstm_mse_vs_sparsity.png', bbox_inches='tight')
    plt.close()

def plot_lstm_mae_vs_sparsity(lstm_df, output_dir):
    """Plot LSTM MAE vs sparsity"""
    fig, ax = plt.subplots(figsize=(12, 8))

    for strategy in sorted(lstm_df['strategy'].unique()):
        data = lstm_df[lstm_df['strategy'] == strategy].sort_values('sparsity')
        ax.plot(data['sparsity_percent'], data['mae'],
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MAE', fontweight='bold')
    ax.set_title('LSTM: Mean Absolute Error vs Sparsity Level', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/lstm_mae_vs_sparsity.png', bbox_inches='tight')
    plt.close()

def plot_lstm_macs_analysis(lstm_df, output_dir):
    """Plot LSTM MACs behavior (special analysis)"""
    fig, ax = plt.subplots(figsize=(12, 8))

    for strategy in sorted(lstm_df['strategy'].unique()):
        data = lstm_df[lstm_df['strategy'] == strategy].sort_values('sparsity')
        ax.plot(data['sparsity_percent'], data['macs'] / 1000,
                'o-', color=COLORS[strategy], label=strategy, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MACs (Thousands)', fontweight='bold')
    ax.set_title('LSTM: Minimal MACs Reduction Despite Pruning', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/lstm_macs_analysis.png', bbox_inches='tight')
    plt.close()

# ============================================================================
# Section 5.4.3: Time-Series Models Comparison
# ============================================================================

def plot_timeseries_mse_comparison(mlp_df, lstm_df, output_dir):
    """Compare MLP vs LSTM MSE performance"""
    fig, ax = plt.subplots(figsize=(12, 8))

    # Use MagnitudeL2 strategy for comparison
    mlp_data = mlp_df[mlp_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    lstm_data = lstm_df[lstm_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')

    ax.plot(mlp_data['sparsity_percent'], mlp_data['mse'],
            'o-', color=COLORS['MLP'], label='MLP', linewidth=3, markersize=10)
    ax.plot(lstm_data['sparsity_percent'], lstm_data['mse'],
            'o-', color=COLORS['LSTM'], label='LSTM', linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MSE', fontweight='bold')
    ax.set_title('Time-Series Models: MSE Comparison (MagnitudeL2 Pruning)', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/timeseries_mse_comparison.png', bbox_inches='tight')
    plt.close()

def plot_timeseries_params_comparison(mlp_df, lstm_df, output_dir):
    """Compare parameter reduction between MLP and LSTM"""
    fig, ax = plt.subplots(figsize=(12, 8))

    # Calculate parameter reduction
    mlp_baseline = mlp_df[mlp_df['sparsity'] == 0.0]['params'].iloc[0]
    lstm_baseline = lstm_df[lstm_df['sparsity'] == 0.0]['params'].iloc[0]

    mlp_data = mlp_df[mlp_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    lstm_data = lstm_df[lstm_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')

    mlp_reduction = (1 - mlp_data['params'] / mlp_baseline) * 100
    lstm_reduction = (1 - lstm_data['params'] / lstm_baseline) * 100

    ax.plot(mlp_data['sparsity_percent'], mlp_reduction,
            'o-', color=COLORS['MLP'], label='MLP', linewidth=3, markersize=10)
    ax.plot(lstm_data['sparsity_percent'], lstm_reduction,
            'o-', color=COLORS['LSTM'], label='LSTM', linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('Parameter Reduction (%)', fontweight='bold')
    ax.set_title('Time-Series Models: Parameter Reduction Comparison', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/timeseries_params_comparison.png', bbox_inches='tight')
    plt.close()

# ============================================================================
# Section 5.5: Cross-Architecture Analysis
# ============================================================================

def plot_strategy_effectiveness_comparison(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir):
    """Compare strategy effectiveness across all architectures at 50% sparsity"""
    fig, ax = plt.subplots(figsize=(14, 8))

    models = ['MobileNetV2', 'ResNet-18', 'MLP', 'LSTM']
    strategies = ['MagnitudeL2', 'Random']

    # Prepare data for 50% sparsity
    data_50 = {
        'MobileNetV2': mobilenet_df[mobilenet_df['sparsity'] == 0.5],
        'ResNet-18': resnet_df[resnet_df['sparsity'] == 0.5],
        'MLP': mlp_df[mlp_df['sparsity'] == 0.5],
        'LSTM': lstm_df[lstm_df['sparsity'] == 0.5]
    }

    baselines = {
        'MobileNetV2': mobilenet_df[mobilenet_df['sparsity'] == 0.0],
        'ResNet-18': resnet_df[resnet_df['sparsity'] == 0.0],
        'MLP': mlp_df[mlp_df['sparsity'] == 0.0],
        'LSTM': lstm_df[lstm_df['sparsity'] == 0.0]
    }

    x = np.arange(len(models))
    width = 0.35

    retention_magnitudeL2 = []
    retention_random = []

    for model in models:
        baseline_data = baselines[model]
        pruned_data = data_50[model]

        if model in ['MobileNetV2', 'ResNet-18']:
            # For classification (higher is better)
            baseline_perf = baseline_data[baseline_data['strategy'] == 'MagnitudeL2']['accuracy'].values[0]
            mag_perf = pruned_data[pruned_data['strategy'] == 'MagnitudeL2']['accuracy'].values[0]
            rand_perf = pruned_data[pruned_data['strategy'] == 'Random']['accuracy'].values[0]

            retention_magnitudeL2.append((mag_perf / baseline_perf) * 100)
            retention_random.append((rand_perf / baseline_perf) * 100)
        else:
            # For regression (lower is better, so inverse)
            baseline_perf = baseline_data[baseline_data['strategy'] == 'MagnitudeL2']['mse'].values[0]
            mag_perf = pruned_data[pruned_data['strategy'] == 'MagnitudeL2']['mse'].values[0]
            rand_perf = pruned_data[pruned_data['strategy'] == 'Random']['mse'].values[0]

            retention_magnitudeL2.append((baseline_perf / mag_perf) * 100)
            retention_random.append((baseline_perf / rand_perf) * 100)

    bars1 = ax.bar(x - width/2, retention_magnitudeL2, width,
                   label='MagnitudeL2', color=COLORS['MagnitudeL2'], alpha=0.8)
    bars2 = ax.bar(x + width/2, retention_random, width,
                   label='Random', color=COLORS['Random'], alpha=0.8)

    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                   f'{height:.1f}%', ha='center', va='bottom', fontweight='bold')

    ax.set_xlabel('Model Architecture', fontweight='bold')
    ax.set_ylabel('Performance Retention (%)', fontweight='bold')
    ax.set_title('Strategy Effectiveness Comparison at 50% Sparsity', fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(models)
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3, axis='y')
    ax.axhline(y=100, color='black', linestyle='--', alpha=0.5, linewidth=2)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/strategy_effectiveness_comparison.png', bbox_inches='tight')
    plt.close()

def plot_model_size_comparison(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir):
    """Compare model sizes across architectures"""
    fig, ax = plt.subplots(figsize=(14, 8))

    models = ['MobileNetV2', 'ResNet-18', 'MLP', 'LSTM']
    sparsity_levels = [0.0, 0.2, 0.5, 0.7]

    data_dict = {
        'MobileNetV2': mobilenet_df,
        'ResNet-18': resnet_df,
        'MLP': mlp_df,
        'LSTM': lstm_df
    }

    x = np.arange(len(models))
    width = 0.2

    for i, sparsity in enumerate(sparsity_levels):
        sizes = []
        for model in models:
            df = data_dict[model]
            size = df[(df['strategy'] == 'MagnitudeL2') &
                     (df['sparsity'] == sparsity)]['size_mb'].values[0]
            sizes.append(size)

        bars = ax.bar(x + i*width, sizes, width,
                      label=f'{int(sparsity*100)}% Sparsity', alpha=0.8)

        # Add value labels on bars
        for bar, size in zip(bars, sizes):
            height = bar.get_height()
            if height > 1:
                label = f'{size:.1f}'
            else:
                label = f'{size:.2f}'
            ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   label, ha='center', va='bottom', fontweight='bold', fontsize=12)

    ax.set_xlabel('Model Architecture', fontweight='bold')
    ax.set_ylabel('Model Size (MB)', fontweight='bold')
    ax.set_title('Model Size Reduction Across Architectures (MagnitudeL2 Pruning)', fontweight='bold')
    ax.set_xticks(x + width * 1.5)
    ax.set_xticklabels(models)
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_yscale('log')

    plt.tight_layout()
    plt.savefig(f'{output_dir}/model_size_comparison.png', bbox_inches='tight')
    plt.close()

def plot_macs_comparison_all_models(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir):
    """Compare MACs across all models"""
    fig, ax = plt.subplots(figsize=(12, 8))

    models_data = [
        (mobilenet_df, 'MobileNetV2', COLORS['MobileNetV2']),
        (resnet_df, 'ResNet-18', COLORS['ResNet-18']),
        (mlp_df, 'MLP', COLORS['MLP']),
        (lstm_df, 'LSTM', COLORS['LSTM'])
    ]

    for df, model_name, color in models_data:
        data = df[df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
        baseline_macs = data[data['sparsity'] == 0.0]['macs'].values[0]

        # Calculate percentage of MACs retained
        macs_retained = (data['macs'] / baseline_macs) * 100

        ax.plot(data['sparsity_percent'], macs_retained, 'o-',
               color=color, label=model_name, linewidth=3, markersize=10)

    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('MACs Retained (%)', fontweight='bold')
    ax.set_title('Computational Cost Reduction Across All Models (MagnitudeL2 Pruning)', fontweight='bold')
    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 105)

    # Add reference lines
    ax.axhline(y=50, color='gray', linestyle=':', alpha=0.5, linewidth=2)
    ax.text(35, 52, '50% MACs', fontsize=14, color='gray', fontweight='bold')

    plt.tight_layout()
    plt.savefig(f'{output_dir}/macs_comparison_all_models.png', bbox_inches='tight')
    plt.close()

# ============================================================================
# Section 5.7: Detailed Results Analysis - Heatmaps
# ============================================================================

def plot_mobilenetv2_strategy_heatmap(mobilenet_df, output_dir):
    """Create strategy comparison heatmap for MobileNetV2"""
    fig, ax = plt.subplots(figsize=(10, 8))

    pivot_data = mobilenet_df.pivot(index='strategy', columns='sparsity_percent', values='accuracy')
    baseline = pivot_data[0.0]
    normalized = (pivot_data.T / baseline * 100).T

    sns.heatmap(normalized, annot=True, fmt='.1f', cmap='RdYlGn',
                center=100, ax=ax, cbar_kws={'label': 'Relative Accuracy (%)'})
    ax.set_title('MobileNetV2: Relative Accuracy Heatmap', fontweight='bold')
    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('Pruning Strategy', fontweight='bold')

    plt.tight_layout()
    plt.savefig(f'{output_dir}/mobilenetv2_strategy_heatmap.png', bbox_inches='tight')
    plt.close()

def plot_resnet18_strategy_heatmap(resnet_df, output_dir):
    """Create strategy comparison heatmap for ResNet-18"""
    fig, ax = plt.subplots(figsize=(10, 8))

    pivot_data = resnet_df.pivot(index='strategy', columns='sparsity_percent', values='accuracy')
    baseline = pivot_data[0.0]
    normalized = (pivot_data.T / baseline * 100).T

    sns.heatmap(normalized, annot=True, fmt='.1f', cmap='RdYlGn',
                center=100, ax=ax, cbar_kws={'label': 'Relative Accuracy (%)'})
    ax.set_title('ResNet-18: Relative Accuracy Heatmap', fontweight='bold')
    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('Pruning Strategy', fontweight='bold')

    plt.tight_layout()
    plt.savefig(f'{output_dir}/resnet18_strategy_heatmap.png', bbox_inches='tight')
    plt.close()

def plot_mlp_strategy_heatmap(mlp_df, output_dir):
    """Create strategy comparison heatmap for MLP"""
    fig, ax = plt.subplots(figsize=(10, 8))

    pivot_data = mlp_df.pivot(index='strategy', columns='sparsity_percent', values='mse')
    baseline = pivot_data[0.0]
    normalized = ((pivot_data.T / baseline - 1) * 100).T

    sns.heatmap(normalized, annot=True, fmt='.1f', cmap='RdYlGn_r',
                center=0, ax=ax, cbar_kws={'label': 'MSE Increase (%)'})
    ax.set_title('MLP: MSE Increase Heatmap', fontweight='bold')
    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('Pruning Strategy', fontweight='bold')

    plt.tight_layout()
    plt.savefig(f'{output_dir}/mlp_strategy_heatmap.png', bbox_inches='tight')
    plt.close()

def plot_lstm_strategy_heatmap(lstm_df, output_dir):
    """Create strategy comparison heatmap for LSTM"""
    fig, ax = plt.subplots(figsize=(10, 8))

    pivot_data = lstm_df.pivot(index='strategy', columns='sparsity_percent', values='mse')
    baseline = pivot_data[0.0]
    normalized = ((pivot_data.T / baseline - 1) * 100).T

    sns.heatmap(normalized, annot=True, fmt='.1f', cmap='RdYlGn_r',
                center=0, ax=ax, cbar_kws={'label': 'MSE Increase (%)'})
    ax.set_title('LSTM: MSE Increase Heatmap', fontweight='bold')
    ax.set_xlabel('Sparsity (%)', fontweight='bold')
    ax.set_ylabel('Pruning Strategy', fontweight='bold')

    plt.tight_layout()
    plt.savefig(f'{output_dir}/lstm_strategy_heatmap.png', bbox_inches='tight')
    plt.close()

# ============================================================================
# Section 5.9: Summary Analysis
# ============================================================================

def create_performance_summary_table(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir):
    """Create comprehensive summary table"""
    fig, ax = plt.subplots(figsize=(16, 12))
    ax.axis('tight')
    ax.axis('off')

    # Prepare data for table - show results at multiple sparsity levels
    table_data = []
    headers = ['Model', 'Strategy', 'Sparsity', 'Baseline', 'Pruned', 'Retention (%)',
               'MACs Red. (%)', 'Size Red. (%)']

    sparsity_levels = [0.2, 0.5, 0.7]

    for df, model_name, metric in [(mobilenet_df, 'MobileNetV2', 'accuracy'),
                                   (resnet_df, 'ResNet-18', 'accuracy'),
                                   (mlp_df, 'MLP', 'mse'),
                                   (lstm_df, 'LSTM', 'mse')]:

        for strategy in ['MagnitudeL2', 'Random']:
            if strategy not in df['strategy'].unique():
                continue

            baseline = df[(df['strategy'] == strategy) & (df['sparsity'] == 0.0)].iloc[0]

            for sparsity in sparsity_levels:
                pruned = df[(df['strategy'] == strategy) & (df['sparsity'] == sparsity)].iloc[0]

                if metric == 'accuracy':
                    baseline_perf = f"{baseline['accuracy']:.2f}%"
                    pruned_perf = f"{pruned['accuracy']:.2f}%"
                    retention = (pruned['accuracy'] / baseline['accuracy']) * 100
                else:  # MSE
                    baseline_perf = f"{baseline['mse']:.2f}"
                    pruned_perf = f"{pruned['mse']:.2f}"
                    retention = (baseline['mse'] / pruned['mse']) * 100

                macs_reduction = (1 - pruned['macs'] / baseline['macs']) * 100
                size_reduction = (1 - pruned['size_mb'] / baseline['size_mb']) * 100

                table_data.append([
                    model_name,
                    strategy,
                    f"{int(sparsity*100)}%",
                    baseline_perf,
                    pruned_perf,
                    f"{retention:.1f}",
                    f"{macs_reduction:.1f}",
                    f"{size_reduction:.1f}"
                ])

    # Create table
    table = ax.table(cellText=table_data, colLabels=headers,
                    cellLoc='center', loc='center',
                    colWidths=[0.12, 0.12, 0.08, 0.12, 0.12, 0.12, 0.12, 0.12])

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.8)

    # Style the table
    for i in range(len(headers)):
        table[(0, i)].set_facecolor('#4CAF50')
        table[(0, i)].set_text_props(weight='bold', color='white')

    # Alternate row colors
    for i in range(1, len(table_data) + 1):
        if i % 2 == 0:
            for j in range(len(headers)):
                table[(i, j)].set_facecolor('#f0f0f0')

    plt.title('Comprehensive Performance Summary Across All Models and Sparsity Levels',
             fontsize=18, fontweight='bold', pad=20)
    plt.savefig(f'{output_dir}/performance_summary_table.png', bbox_inches='tight')
    plt.close()

def plot_optimal_points_analysis(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir):
    """Identify and plot optimal operating points for each model"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

    # MobileNetV2
    mobilenet_data = mobilenet_df[mobilenet_df['strategy'] == 'MagnitudeL2'].sort_values('macs_millions')
    ax1.scatter(mobilenet_data['macs_millions'], mobilenet_data['accuracy'],
               color=COLORS['MobileNetV2'], s=150, alpha=0.8, edgecolors='black', linewidth=1.5)
    ax1.plot(mobilenet_data['macs_millions'], mobilenet_data['accuracy'],
            color=COLORS['MobileNetV2'], linestyle='--', alpha=0.7, linewidth=2)
    ax1.set_xlabel('MACs (Millions)', fontweight='bold')
    ax1.set_ylabel('Accuracy (%)', fontweight='bold')
    ax1.set_title('MobileNetV2: Optimal Operating Points', fontweight='bold')
    ax1.grid(True, alpha=0.3)

    # ResNet-18
    resnet_data = resnet_df[resnet_df['strategy'] == 'MagnitudeL2'].sort_values('macs_millions')
    ax2.scatter(resnet_data['macs_millions'], resnet_data['accuracy'],
               color=COLORS['ResNet-18'], s=150, alpha=0.8, edgecolors='black', linewidth=1.5)
    ax2.plot(resnet_data['macs_millions'], resnet_data['accuracy'],
            color=COLORS['ResNet-18'], linestyle='--', alpha=0.7, linewidth=2)
    ax2.set_xlabel('MACs (Millions)', fontweight='bold')
    ax2.set_ylabel('Accuracy (%)', fontweight='bold')
    ax2.set_title('ResNet-18: Optimal Operating Points', fontweight='bold')
    ax2.grid(True, alpha=0.3)

    # MLP
    mlp_data = mlp_df[mlp_df['strategy'] == 'MagnitudeL2'].sort_values('macs')
    ax3.scatter(mlp_data['macs'] / 1000, mlp_data['mse'],
               color=COLORS['MLP'], s=150, alpha=0.8, edgecolors='black', linewidth=1.5)
    ax3.plot(mlp_data['macs'] / 1000, mlp_data['mse'],
            color=COLORS['MLP'], linestyle='--', alpha=0.7, linewidth=2)
    ax3.set_xlabel('MACs (Thousands)', fontweight='bold')
    ax3.set_ylabel('MSE', fontweight='bold')
    ax3.set_title('MLP: Optimal Operating Points', fontweight='bold')
    ax3.grid(True, alpha=0.3)

    # LSTM
    lstm_data = lstm_df[lstm_df['strategy'] == 'MagnitudeL2'].sort_values('macs')
    ax4.scatter(lstm_data['macs'] / 1000, lstm_data['mse'],
               color=COLORS['LSTM'], s=150, alpha=0.8, edgecolors='black', linewidth=1.5)
    ax4.plot(lstm_data['macs'] / 1000, lstm_data['mse'],
            color=COLORS['LSTM'], linestyle='--', alpha=0.7, linewidth=2)
    ax4.set_xlabel('MACs (Thousands)', fontweight='bold')
    ax4.set_ylabel('MSE', fontweight='bold')
    ax4.set_title('LSTM: Optimal Operating Points', fontweight='bold')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/optimal_points_analysis.png', bbox_inches='tight')
    plt.close()

# ============================================================================
# Main execution function
# ============================================================================

def main():
    """Generate all plots for Chapter 5"""
    print("Generating Chapter 5 plots...")

    # Load data
    mobilenet_results, resnet_results, mlp_results, lstm_results = load_all_results()

    if any(x is None for x in [mobilenet_results, resnet_results, mlp_results, lstm_results]):
        print("Error: Could not load all result files. Please check file names and paths.")
        return

    # Convert to DataFrames
    mobilenet_df = results_to_dataframe(mobilenet_results, 'MobileNetV2', 'classification')
    resnet_df = results_to_dataframe(resnet_results, 'ResNet-18', 'classification')
    mlp_df = results_to_dataframe(mlp_results, 'MLP', 'regression')
    lstm_df = results_to_dataframe(lstm_results, 'LSTM', 'regression')

    # Create output directory
    output_dir = create_output_directory()

    print(f"Saving plots to: {output_dir}")

    # Section 5.3.1: MobileNetV2 Analysis
    print("Generating MobileNetV2 plots...")
    plot_mobilenetv2_accuracy_vs_sparsity(mobilenet_df, output_dir)
    plot_mobilenetv2_macs_reduction(mobilenet_df, output_dir)
    plot_mobilenetv2_size_reduction(mobilenet_df, output_dir)
    plot_mobilenetv2_pareto_frontier(mobilenet_df, output_dir)

    # Section 5.3.2: ResNet-18 Analysis
    print("Generating ResNet-18 plots...")
    plot_resnet18_accuracy_vs_sparsity(resnet_df, output_dir)
    plot_resnet18_macs_reduction(resnet_df, output_dir)
    plot_resnet18_pareto_frontier(resnet_df, output_dir)

    # Section 5.3.3: CNN Comparison
    print("Generating CNN comparison plots...")
    plot_cnn_accuracy_comparison(mobilenet_df, resnet_df, output_dir)
    plot_cnn_efficiency_comparison(mobilenet_df, resnet_df, output_dir)

    # Section 5.4.1: MLP Analysis
    print("Generating MLP plots...")
    plot_mlp_mse_vs_sparsity(mlp_df, output_dir)
    plot_mlp_mae_vs_sparsity(mlp_df, output_dir)
    plot_mlp_macs_reduction(mlp_df, output_dir)

    # Section 5.4.2: LSTM Analysis
    print("Generating LSTM plots...")
    plot_lstm_mse_vs_sparsity(lstm_df, output_dir)
    plot_lstm_mae_vs_sparsity(lstm_df, output_dir)
    plot_lstm_macs_analysis(lstm_df, output_dir)

    # Section 5.4.3: Time-series Comparison
    print("Generating time-series comparison plots...")
    plot_timeseries_mse_comparison(mlp_df, lstm_df, output_dir)
    plot_timeseries_params_comparison(mlp_df, lstm_df, output_dir)

    # Section 5.5: Cross-Architecture Analysis
    print("Generating cross-architecture plots...")
    plot_strategy_effectiveness_comparison(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir)
    plot_model_size_comparison(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir)
    plot_macs_comparison_all_models(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir)

    # Section 5.7: Heatmaps
    print("Generating strategy heatmaps...")
    plot_mobilenetv2_strategy_heatmap(mobilenet_df, output_dir)
    plot_resnet18_strategy_heatmap(resnet_df, output_dir)
    plot_mlp_strategy_heatmap(mlp_df, output_dir)
    plot_lstm_strategy_heatmap(lstm_df, output_dir)

    # Section 5.9: Summary Analysis
    print("Generating summary plots...")
    create_performance_summary_table(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir)
    plot_optimal_points_analysis(mobilenet_df, resnet_df, mlp_df, lstm_df, output_dir)

    print(f"\n✅ All plots generated successfully!")
    print(f"📁 Total plots created: 26")
    print(f"📁 Saved to directory: {os.path.abspath(output_dir)}")

    # Print summary of generated plots
    plot_files = [f for f in os.listdir(output_dir) if f.endswith('.png')]
    print(f"\nGenerated plots:")
    for i, plot_file in enumerate(sorted(plot_files), 1):
        print(f"{i:2d}. {plot_file}")

if __name__ == "__main__":
    main()

Generating Chapter 5 plots...
Saving plots to: chapter5_plots
Generating MobileNetV2 plots...
Generating ResNet-18 plots...
Generating CNN comparison plots...
Generating MLP plots...
Generating LSTM plots...
Generating time-series comparison plots...
Generating cross-architecture plots...
Generating strategy heatmaps...
Generating summary plots...

✅ All plots generated successfully!
📁 Total plots created: 26
📁 Saved to directory: /home/muis/thesis/github-repo/master-thesis/utils/chapter5_plots

Generated plots:
 1. cnn_accuracy_comparison.png
 2. cnn_efficiency_comparison.png
 3. lstm_macs_analysis.png
 4. lstm_mae_vs_sparsity.png
 5. lstm_mse_vs_sparsity.png
 6. lstm_strategy_heatmap.png
 7. macs_comparison_all_models.png
 8. mlp_macs_reduction.png
 9. mlp_mae_vs_sparsity.png
10. mlp_mse_vs_sparsity.png
11. mlp_strategy_heatmap.png
12. mobilenetv2_accuracy_vs_sparsity.png
13. mobilenetv2_macs_reduction.png
14. mobilenetv2_pareto_frontier.png
15. mobilenetv2_size_reduction.png
16. mob